In [2]:
import json
from elasticsearch import Elasticsearch

In [3]:
with open('../credentials.json') as f:
    credentials = json.load(f)

bonsai_url = credentials['bonsai_url']
access_key = credentials['access_key']
access_secret = credentials['access_secret']


In [4]:
es = Elasticsearch(
    [bonsai_url],
    basic_auth=(access_key, access_secret),
    headers={'Content-Type': 'application/json'}
)
info = es.info()
version_number = info['version']['number']

print(f"Elasticsearch version: {version_number}")

Elasticsearch version: 7.2.0


In [5]:
terms = [1,2,3,4,5,6,8]

# Construct the script source as a string
script_source = """
double count = 0.0; 
for (long term : params.terms) { 
  if (doc['skill_ids'].contains(term)) { 
    count++; 
    } 
  } 
return count/ (double) params.terms.length;
"""

# Construct the query using the terms variable and script_source
query = {
  "query": {
    "function_score": {
      "query": {
        "terms": {
          "skill_ids": terms
        }
      },
      "functions": [
        {
          "script_score": {
            "script": {
              "source": script_source,
              "params": {
                "terms": terms
              }
            }
          }
        }
      ]
    }
  }
}

In [6]:
index_name = "learning_opportunities_production_20230711123025421"

In [7]:
response = es.search(index=index_name, body=query)

In [8]:
for elem in response['hits']['hits']:
    print(elem['_source']['course'], elem['_source']['skill_ids'], elem['_score'])

Training from the BACK of the Room Practitioner – Virtual Edition, Deutsch/Mai [8, 12, 7, 5, 1, 2, 3] 0.71428573
Training from the BACK of the Room Practitioner - Virtual Edition, English [3, 2, 1, 5, 7, 12, 8] 0.71428573
Training from the BACK of the Room! (TBR) Practitioner class, 2 Tage Präsenzkurs [3, 2, 1, 5, 7, 12, 8] 0.71428573
Training from the BACK of the Room Practitioner – Virtual Edition, Deutsch [3, 2, 1, 5, 7, 12, 8] 0.71428573
Sustainable investment: FFPC session [2, 6, 1, 5, 3, 9] 0.71428573
CAS Diskurskompetenzen für Führungskräfte [2, 5, 4, 3, 8, 7] 0.71428573
CAS Performance und Compensation Management [4, 5, 6, 7, 8, 9, 10] 0.5714286
CAS HR-Marketing [1, 3, 6, 8, 9] 0.5714286
CAS Human Capital & Leadership [1, 3, 4, 7, 8, 10, 11] 0.5714286
CAS Consulting, Communication & Culture [2, 3, 5, 8] 0.5714286


In [17]:
evrlearn_job_profiles_path = "../data/raw/evrlearn/job_profiles.json"
evrlearn_job_skills_path = "../data/raw/evrlearn/job_skills.json"

In [18]:
job_profiles = json.load(open(evrlearn_job_profiles_path, "r"))
job_skills = json.load(open(evrlearn_job_skills_path, "r"))

In [23]:
job_skills

[{'id': 1,
  'name': 'Kommunikation',
  'skill_group': 'Personal skills',
  'skill_group_id': 1},
 {'id': 2,
  'name': 'Selbstmanagement/Führen der eigenen Person',
  'skill_group': 'Personal skills',
  'skill_group_id': 1},
 {'id': 3,
  'name': 'Soziale Kompetenzen',
  'skill_group': 'Personal skills',
  'skill_group_id': 1},
 {'id': 4,
  'name': 'Gestaltung und Problemlösung',
  'skill_group': 'Personal skills',
  'skill_group_id': 1},
 {'id': 5,
  'name': 'Umgang mit Komplexität',
  'skill_group': 'Personal skills',
  'skill_group_id': 1},
 {'id': 6,
  'name': 'Abstraktionsvermögen',
  'skill_group': 'Personal skills',
  'skill_group_id': 1},
 {'id': 7,
  'name': 'Betriebswirtschaftliche Zusammenhänge',
  'skill_group': 'Social skills',
  'skill_group_id': 2},
 {'id': 8,
  'name': 'Management der ICT',
  'skill_group': 'Professional skills',
  'skill_group_id': 3},
 {'id': 9,
  'name': 'Projektmanagement',
  'skill_group': 'Professional skills',
  'skill_group_id': 3},
 {'id': 10,
 

In [30]:
job_profiles[7]

{'id': 8,
 'name': 'ICT System Specialist',
 'level_id': 3,
 'level': 'Senior',
 'job_profile_skills': [{'id': 113,
   'job_profile_id': 8,
   'job_skill_id': 1,
   'level_id': 1,
   'created_at': '2023-07-25T09:51:25.050+02:00',
   'updated_at': '2023-07-25T09:51:25.050+02:00',
   'certificate': False},
  {'id': 114,
   'job_profile_id': 8,
   'job_skill_id': 2,
   'level_id': 2,
   'created_at': '2023-07-25T09:51:25.052+02:00',
   'updated_at': '2023-07-25T09:51:25.052+02:00',
   'certificate': False},
  {'id': 115,
   'job_profile_id': 8,
   'job_skill_id': 3,
   'level_id': 2,
   'created_at': '2023-07-25T09:51:25.054+02:00',
   'updated_at': '2023-07-25T09:51:25.054+02:00',
   'certificate': False},
  {'id': 116,
   'job_profile_id': 8,
   'job_skill_id': 4,
   'level_id': 3,
   'created_at': '2023-07-25T09:51:25.056+02:00',
   'updated_at': '2023-07-25T09:51:25.056+02:00',
   'certificate': False},
  {'id': 117,
   'job_profile_id': 8,
   'job_skill_id': 5,
   'level_id': 2,
   '